In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [ ]:
tr_mcc_codes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/tr_mcc_codes.csv', sep=';')
tr_types = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/tr_types.csv', sep=';')
transactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/transactions.csv', nrows=1000000)
gender_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/gender_train.csv')


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
transaction_sample = transactions['tr_type'].sample(n=1000, ignore_index=True)

# Найдем подходящие нам коды операций
trans_types = tr_types[tr_types['tr_description'].str.contains('АТМ|POS')]

print(f"Доля транзакций в выборке - {sum(transaction_sample.isin(trans_types['tr_type'])) / len(transaction_sample)}")

Доля транзакций в выборке - 0.608


## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
frequency_tr_type = transactions['tr_type'].value_counts()
top_10 = tr_types[tr_types['tr_type'].isin(frequency_tr_type[:10].index)]
print(top_10)

     tr_type                                     tr_description
39      7010       Взнос наличных через АТМ (в своем тер.банке)
47      7030  Перевод на карту (с карты) через АТМ (в предел...
59      7070  Перевод на карту (с карты) через Мобильный бан...
60      7071  Перевод на карту (с карты) через Мобильный бан...
98      1010                              Покупка. POS ТУ СБ РФ
99      1030                     Оплата услуги. Банкоматы СБ РФ
100     1100                                Покупка. ТУ  Россия
101     1110                             Покупка. POS ТУ Россия
106     2010              Выдача наличных в АТМ Сбербанк России
129     2370  Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
incomes = transactions[transactions['amount'] > 0].groupby(['customer_id'])['amount'].sum()
outgoings = transactions[transactions['amount'] < 0].groupby(['customer_id'])['amount'].sum()

id_max = incomes.idxmax()
id_min = outgoings.idxmin()

print(f"1) {id_max}")
print(f"2) {id_min}")
print(f"3) {abs(incomes.get(incomes.idxmax()) + outgoings.get(outgoings.idxmin()))}")

1) 70780820
2) 70780820
3) 1837317.980000019


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
indexes = top_10['tr_type'].values
type_top = transactions[transactions['tr_type'].isin(indexes)].groupby(['tr_type'])['amount']
print(round(pd.concat([type_top.median(), type_top.mean()], keys=["Median", "Mean"], axis=1), 2))
print()

types_of_min_max = transactions[transactions['customer_id'].isin((id_max, id_min))].groupby(['tr_type'])['amount']
print(round(pd.concat([types_of_min_max.median(), types_of_min_max.mean()], keys=["Median", "Mean"], axis=1), 2))

            Median       Mean
tr_type                      
1010      -7411.52  -19784.75
1030      -2245.92   -5320.98
1100     -10188.26  -44061.83
1110     -11207.57  -32119.33
2010     -44918.32 -136077.63
2370     -44918.32 -205418.25
7010     112295.79  276391.79
7030      13951.52   86104.33
7070      11319.42   65569.83
7071       3593.47   66806.83

             Median        Mean
tr_type                        
2010    -3368873.66 -2941055.87
2330    -2245915.77 -2382398.34
2370    -2245915.77 -2218504.99
6110       10028.01    17562.93
7020       10319.98    14651.29
7021       28500.67    28500.67
7030        8848.91    14061.96
7031       12992.62    19210.36
7034       14306.48    14306.48
7040       14396.32    21538.33
7070        8714.15    13848.21
7071       12543.44    18600.57
7074        9387.93    22811.59


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
incomes_mean_by_gender = transactions[transactions['amount'] > 0].groupby(['gender'])['amount'].mean()
outgoings_mean_by_gender = transactions[transactions['amount'] < 0 ].groupby(['gender'])['amount'].mean()

print(f"1) {round(abs(outgoings_mean_by_gender.get(0) - outgoings_mean_by_gender.get(1)), 2)}")
print(f"1) {round(abs(incomes_mean_by_gender.get(0) - incomes_mean_by_gender.get(1)), 2)}")

1) 32718.05
1) 63366.57


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
men_incomes = transactions[['gender', 'tr_type', 'amount']][(transactions['amount'] > 0)
 & (transactions['gender'] == 0)]

women_incomes = transactions[['gender', 'tr_type', 'amount']][(transactions['amount'] > 0)
 & (transactions['gender'] == 1)]

men_max_income = round(men_incomes.groupby('tr_type')['amount'].sum().sort_values(ascending=False)[-10:], 2).rename('max_income')
women_max_income = round(women_incomes.groupby('tr_type')['amount'].sum().sort_values(ascending=False)[-10:], 2).rename('max_income')

print(men_max_income)
print()
print(women_max_income)
print()


print(pd.Series(men_max_income.index.intersection(women_max_income.index), name='same_types'))

tr_type
1210    83644.87
2210    68315.82
4100    40334.88
1010    31532.66
2110    22459.16
7075     6737.75
2370     2335.75
4210     2245.92
4110     2245.92
4051     1122.96
Name: max_income, dtype: float64

tr_type
2010    224591.59
2110    179673.26
7015    121279.45
2370    113194.15
7041     87590.72
6000     79280.83
8100     64682.37
1010     35317.03
4110      4491.84
2020      2245.92
Name: max_income, dtype: float64

0    1010
1    2110
2    2370
3    4110
Name: same_types, dtype: int64


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
mcc_outgoings = transactions[transactions['amount'] < 0]
mcc_outgoings = mcc_outgoings.groupby(['gender', 'mcc_code', 'mcc_description'])['amount'].sum()
print(mcc_outgoings)
print()

relative_difference = round(abs(mcc_outgoings.get(0) - mcc_outgoings.get(1)).dropna().rename('difference'), 2)
print(relative_difference.sort_values(ascending=False)[:10])

gender  mcc_code  mcc_description                                                            
0.0     742       Ветеринарные услуги                                                             -386207.68
        1711      Генеральные подрядчики по вентиляции, теплоснабжению, и водопроводу             -674897.69
        1799      Подрядчики, специализированная торговля — нигде более не классифицированные     -261626.73
        2741      Разнообразные издательства/печатное дело                                         -20746.42
        3000      Авиалинии, авиакомпании                                                       -74431645.27
                                                                                                    ...     
1.0     8999      Профессиональные услуги, нигде ранее не классифицируемые                      -28643346.63
        9222      Штрафы                                                                          -988205.17
        9311      Налоговые платеж

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [ ]:
transactions['tr_hour'] = transactions['tr_datetime'].map(lambda x: int(x.split()[1].split(':')[0]))
print(transactions['tr_hour'])
print()

negative_night_transactions = transactions[(transactions['tr_hour'] >= 0) & (transactions['tr_hour'] <= 6) & (transactions['amount'] < 0)]

print(f"Количетсво транзакций для мужчин в ночное время: {len(negative_night_transactions[negative_night_transactions['gender'] == 0])}")
print(f"Количетсво транзакций для женщин в ночное время: {len(negative_night_transactions[negative_night_transactions['gender'] == 1])}")

0         10
1          7
2          7
3          8
4         14
          ..
999579    23
999580    19
999581    22
999582    23
999583    23
Name: tr_hour, Length: 999584, dtype: int64

Количетсво транзакций для мужчин в ночное время: 43510
Количетсво транзакций для женщин в ночное время: 46694


## Доп. задание
Сформируйте датафрейм, в котором будет 24 строки. Каждая строка это час в сутках. Столбцы в датафрейме такие: 1) номер часа, 2) код mcc-категории, по которой в этот час больше всего транзакций, 3) описание этой mcc-категории Какая mcc-категория встречается чаще других?

In [ ]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_hour
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,10
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14


In [ ]:
transactions_per_hour_by_mcc = transactions[['tr_hour', 'mcc_code', 'mcc_description']].groupby('tr_hour').value_counts()

transactions_per_hour = transactions_per_hour_by_mcc.groupby('tr_hour').max()

transactions_per_hour

tr_hour
0     36399
1      4150
2      5912
3      6465
4      6587
5      7366
6      8690
7      8915
8     12112
9     13417
10    15548
11    16109
12    17250
13    16308
14    15175
15    14652
16    13763
17    13566
18    11408
19     8825
20     6507
21     4364
22     3425
23     2147
dtype: int64

In [ ]:
transactions_per_hour_by_mcc = transactions_per_hour_by_mcc.reset_index(name='count')
transactions_per_hour = transactions_per_hour.reset_index(name='count')

result = pd.merge(transactions_per_hour_by_mcc, transactions_per_hour)
result = result[['mcc_code', 'mcc_description']]

In [ ]:
result

,mcc_code,mcc_description
0,5411,"Бакалейные магазины, супермаркеты"
1,6010,Финансовые институты — снятие наличности вручную
2,6010,Финансовые институты — снятие наличности вручную
3,6010,Финансовые институты — снятие наличности вручную
4,6010,Финансовые институты — снятие наличности вручную
5,6010,Финансовые институты — снятие наличности вручную
6,6010,Финансовые институты — снятие наличности вручную
7,6010,Финансовые институты — снятие наличности вручную
8,6011,Финансовые институты — снятие наличности автом...
9,6011,Финансовые институты — снятие наличности автом...


In [ ]:
result['mcc_code'].reset_index().groupby('mcc_code').count().rename(columns={'index': 'Counter'}).sort_values(by=['Counter'], ascending=False)

,Counter
mcc_code,
6010,9
6011,9
4814,3
5411,3
